In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from datetime import datetime
from sklearn.linear_model import SGDClassifier
import sklearn.cross_validation
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost
from vowpalwabbit.sklearn_vw import VWClassifier
import langdetect as ld
#from difflib import SequenceMatcher
from fuzzywuzzy import fuzz

In [3]:
# SOURCE = "../data/google_sample.csv"
# df = pd.read_csv(SOURCE, sep=',')

# SOURCE = "../data/valid_set.csv"
# df = pd.read_csv(SOURCE, sep=',', index_col=0)

SOURCE = "../data/train3_updated.csv"
df = pd.read_csv(SOURCE, sep=',')

# SOURCE = "../data/arzam_test.csv"
# df = pd.read_csv(SOURCE, sep=',')

In [5]:
df.head()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
0,7,0.005422,0,1000.0,3,3,0,2,5,15,0.250000,5,22,93,0,0,1,1
1,1,0.012844,0,77170.0,3,3,0,2,5,15,0.250000,5,22,66,0,0,1,0
2,1,0.002102,0,1790.0,3,2,0,2,5,21,0.192308,3,10,39,84,0,1,0
3,7,0.013889,0,3000.0,3,2,0,2,5,21,0.192308,3,10,48,84,0,1,1
4,1,0.033885,1,5470.0,3,4,0,1,1,7,0.125000,3,10,65,59,15,1,1


In [6]:
df.shape

(5150998, 18)

In [4]:
df.SearchDate.isnull().values.any()

False

In [46]:
# df.SearchCategoryLevel.value_counts()

3    4398631
2     942472
1     425039
Name: SearchCategoryLevel, dtype: int64

In [27]:
# df.isnull()
df.SearchQuery.fillna("", inplace=True)
df.Title.fillna("", inplace=True)
df.SearchParams.fillna("", inplace=True)

## Feature Engineering

In [6]:
df["SearchDate"] = pd.to_datetime(df["SearchDate"], format='%Y-%m-%d', errors='coerce')

df["day_of_week"] = df["SearchDate"].dt.dayofweek
df["hour"] = df["SearchDate"].dt.hour

# df["AdTitleSZ"] = df["Title"].str.len()
# df["SearchParamsSZ"] = df["SearchParams"].str.len()
# df["SearchQuerySZ"] = df["SearchQuery"].str.len()

In [12]:
def getSearchParamsNum(x):
    x['SearchParamsNum'] = len(str(x["SearchParams"]).split(":")) - 1
    return x

df.apply(getSearchParamsNum, axis=1)

,SearchID,AdID,Position,HistCTR,IsClick,SearchDate,IPID,UserID,IsUserLoggedOn,SearchQuery,SearchLocationID,SearchCategoryID,SearchParams,SearchLocationLevel,SearchRegionID,SearchCityID,SearchCategoryLevel,SearchParentCategoryID,SearchSubcategoryID,UserAgentID,UserAgentOSID,UserDeviceID,UserAgentFamilyID,AdCategoryID,Params,Price,Title,AdCategoryLevel,AdParentCategoryID,AdSubcategoryID,weight,reg_ads,highlighted_ads,context_ads,date,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,SearchParamsNum
15,662,3259133,7,0.009569,0,2015-05-17 21:10:38,1606142,1713403,1,,813,44,,3,53.0,1159.0,3,12,22,52255,7,2014,64,44.0,{223:'Оборудование и аксессуары'},1342.0,Описание Штатив Polaroid 42 Travel,3.0,12.0,22.0,1,3,0,2,2015-05-17,6,21,48,0,0,0
16,662,22754496,1,0.011035,1,2015-05-17 21:10:38,1606142,1713403,1,,813,44,,3,53.0,1159.0,3,12,22,52255,7,2014,64,44.0,{223:'Компактные фотоаппараты'},2500.0,"Экшн-камера, аналог GoPro",3.0,12.0,22.0,1,3,0,2,2015-05-17,6,21,41,0,0,0
28,1672,32026171,7,0.005291,0,2015-05-17 18:03:59,1274955,665872,0,,1344,29,{483:'Акустика'},3,51.0,3191.0,3,12,57,16119,20,2014,62,29.0,{483:'Акустика'},1000.0,Колонки Defender I-Wave S16 2x4 Вт RMS,3.0,12.0,57.0,1,3,0,2,2015-05-17,6,18,47,24,0,1
29,1672,33817299,1,0.031800,1,2015-05-17 18:03:59,1274955,665872,0,,1344,29,{483:'Акустика'},3,51.0,3191.0,3,12,57,16119,20,2014,62,29.0,{483:'Акустика'},1409.0,Колонки Defender 2.1 Blaze S12 (2х3+6)Вт,3.0,12.0,57.0,1,3,0,2,2015-05-17,6,18,50,24,0,1
36,2590,5184150,7,0.008260,1,2015-05-17 15:51:21,973137,3810498,0,,547,2,,3,19.0,2115.0,3,12,5,30644,43,2014,25,2.0,NaN,15990.0,Lenovo G5030 (черный),3.0,12.0,5.0,1,2,1,2,2015-05-17,6,15,27,0,0,0
37,2590,31565576,1,0.035608,0,2015-05-17 15:51:21,973137,3810498,0,,547,2,,3,19.0,2115.0,3,12,5,30644,43,2014,25,2.0,NaN,1999.0,Ноутбук ASUS F553MA-BING-SX394B,3.0,12.0,5.0,1,2,1,2,2015-05-17,6,15,38,0,0,0
39,2655,6892762,1,0.023745,1,2015-05-18 17:03:05,964623,2023514,1,,1944,26,{132:'Телевизоры и проекторы'},3,52.0,1056.0,3,12,29,1027,20,2014,7,26.0,{132:'Телевизоры и проекторы'},7989.0,LED Телевизор Samsung LT19C350EX,3.0,12.0,29.0,1,2,1,2,2015-05-18,0,17,41,50,0,1
40,2655,24438982,7,0.008596,0,2015-05-18 17:03:05,964623,2023514,1,,1944,26,{132:'Телевизоры и проекторы'},3,52.0,1056.0,3,12,29,1027,20,2014,7,26.0,{132:'Телевизоры и проекторы'},3500.0,3D-проектор,3.0,12.0,29.0,1,2,1,2,2015-05-18,0,17,19,50,0,1
41,2702,23094697,1,0.013514,1,2015-05-17 09:45:57,1363717,1064501,0,спортивная одежда,3953,22,{175:'Мужская одежда'},2,47.0,1974.0,3,3,47,33735,7,2014,25,22.0,"{175:'Мужская одежда', 176:'Брюки'}",1679.0,Брюки спортивные,3.0,3.0,47.0,1,3,1,1,2015-05-17,6,9,31,35,33,1
42,2764,21694989,1,0.017794,1,2015-05-17 20:23:24,1371699,3691786,0,вечерние платья,3960,0,,2,18.0,623.0,1,10,45,22293,20,2014,25,22.0,"{83:'Другое', 175:'Женская одежда'}",1750.0,Sale! Кружевной комбинезон -50%,3.0,3.0,47.0,1,0,1,1,2015-05-17,6,20,50,0,29,0


In [94]:
def calculateAdQueryRatios(x):
    searchAdSimpleRatio = fuzz.ratio(str(x["Title"]), str(x["SearchQuery"]))
    searchAdPartialRatio = fuzz.partial_ratio(str(x["Title"]), str(x["SearchQuery"]))
    return searchAdSimpleRatio, searchAdPartialRatio

df["SearchAdSimpleRatio"], df["SearchAdPartialRatio"] = zip(*df.apply(calculateAdQueryRatios, axis=1))

In [95]:
df.shape
# df.head()

(49, 33)

In [40]:
any(c.isalpha() for c in "4.")

False

In [ ]:
def detectLanguage(x):
    
    query = ""
    
    if (str(x["SearchQuery"]) != "" or (any(c.isalpha() for c in query) == False)):
        query = "No text found here."
# #         print("hello")
#         query = "No text found here." #Need some text here otherwise will get errors from ld
#     elif (any(c.isalpha() for c in query) == False):
#         query = "No text found here."
#     else:
# #         print("bye")
    else:
        query = str(x["SearchQuery"])
            
#     print("query:" + query)
    
    detectedLang = ld.detect(query.decode('utf-8'))
    if detectedLang == "ru":
        x['SearchRussian'] = 1
    else:
        x['SearchRussian'] = 0
        
    return x

df.apply(detectLanguage, axis=1)

# query=""
# ld.detect(query.decode('utf-8'))

In [81]:
for chunk in pd.read_csv(SOURCE, sep=',', chunksize=10, usecols=["SearchID", "AdID", "SearchQuery"]):
    chunk = chunk.apply(detectLanguage , axis=1)
    
    chunk.SearchQuery.fillna(0, inplace=True)
    chunk.AdID.fillna(0, inplace=True)
    chunk.SearchID.fillna(0, inplace=True)
    
    tempDf = pd.merge(df, chunk)
    mergedDf = mergedDf.append(tempDf)
    
df = mergedDf

In [82]:
df.shape

(49, 32)

In [87]:
df.head(49)

,SearchID,AdID,Position,HistCTR,IsClick,SearchDate,IPID,UserID,IsUserLoggedOn,SearchQuery,SearchLocationID,SearchCategoryID,SearchParams,SearchLocationLevel,SearchRegionID,SearchCityID,SearchCategoryLevel,SearchParentCategoryID,SearchSubcategoryID,UserAgentID,UserAgentOSID,UserDeviceID,UserAgentFamilyID,AdCategoryID,Params,Price,Title,AdCategoryLevel,AdParentCategoryID,AdSubcategoryID,weight,SearchRussian
0,121,1220223,7,0.005422,1,2015-05-09 22:21:40.0,490755,3817669,0,0,2685,44,NaN,3,9.0,1582.0,3,12,22,1582,44,2014,62,44.0,{223:'Бинокли и телескопы'},1000.0,"Лупа,монокуляр на очковой оправе,Отправка по Р...",3.0,12.0,22.0,1,0
1,121,17114456,1,0.012844,0,2015-05-09 22:21:40.0,490755,3817669,0,0,2685,44,NaN,3,9.0,1582.0,3,12,22,1582,44,2014,62,44.0,{223:'Зеркальные фотоаппараты'},77170.0,Зеркальный фотоаппарат Canon EOS 6D (WG) Body,3.0,12.0,22.0,1,0
2,181,6040959,1,0.002102,0,2015-04-30 10:24:49.0,1507803,2391097,0,0,230,22,"{83:'Платья и юбки', 175:'Женская одежда', 90:...",3,2.0,2951.0,3,3,47,63091,20,2014,25,22.0,"{83:'Платья и юбки', 175:'Женская одежда'}",1790.0,Юбка LOST INK с доставкой,3.0,3.0,47.0,1,0
3,181,13840581,7,0.013889,1,2015-04-30 10:24:49.0,1507803,2391097,0,0,230,22,"{83:'Платья и юбки', 175:'Женская одежда', 90:...",3,2.0,2951.0,3,3,47,63091,20,2014,25,22.0,"{83:'Платья и юбки', 175:'Женская одежда'}",3000.0,Брендовые вечерние платья,3.0,3.0,47.0,1,0
4,198,14515103,1,0.033885,1,2015-05-07 10:46:21.0,951832,372668,1,chevrolet cruze,3960,34,"{598:'Для автомобилей', 5:'Запчасти'}",2,18.0,623.0,3,6,7,54509,44,2014,62,34.0,"{5:'Запчасти', 598:'Для автомобилей'}",5470.0,Запчасти б/у и новые для chevrolet cruze (09-),3.0,6.0,7.0,1,0
5,404,9840503,7,0.012311,0,2015-04-30 16:48:21.0,130853,2621226,0,0,2473,49,NaN,3,14.0,698.0,2,8,45,18318,20,2014,25,46.0,NaN,317.0,Муравьиная ферма оптом и в розницу - акция,3.0,8.0,40.0,1,0
6,404,11801992,1,0.007371,1,2015-04-30 16:48:21.0,130853,2621226,0,0,2473,49,NaN,3,14.0,698.0,2,8,45,18318,20,2014,25,46.0,NaN,10419.0,Инкубатор Блиц 48 цифровой,3.0,8.0,40.0,1,0
7,438,20069581,7,0.001996,0,2015-05-01 15:48:08.0,599651,2675985,1,0,2060,35,NaN,2,44.0,1890.0,3,9,50,10275,20,2014,25,35.0,NaN,620.0,Горшок для цветов «Чашка»,3.0,9.0,50.0,1,0
8,438,29135166,1,0.004846,1,2015-05-01 15:48:08.0,599651,2675985,1,0,2060,35,NaN,2,44.0,1890.0,3,9,50,10275,20,2014,25,35.0,NaN,878.0,Аэросад Луковое счастье,3.0,9.0,50.0,1,0
9,466,6955209,7,0.007163,1,2015-05-11 21:41:25.0,1558246,991573,0,0,547,41,{143:'iPhone'},3,19.0,2115.0,3,12,42,24424,20,2014,64,41.0,{143:'iPhone'},450.0,Автодержатель для iPhone с креплением на руль!,3.0,12.0,42.0,1,0


In [8]:
TITLE = ['Position', 'HistCTR', 'IsUserLoggedOn', 'Price', 'day_of_week', 'hour', 'AdTitleSZ', 'SearchParamsSZ', 'SearchQuerySZ', 'IsClick']
inputs = df[TITLE].drop(["IsClick"], axis=1)
inputs.shape
inputs.head()

,Position,HistCTR,IsUserLoggedOn,Price,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
0,7,0.005422,0,1000.0,5,22,93.0,NaN,NaN
1,1,0.012844,0,77170.0,5,22,66.0,NaN,NaN
2,1,0.002102,0,1790.0,3,10,39.0,84.0,NaN
3,7,0.013889,0,3000.0,3,10,48.0,84.0,NaN
4,1,0.033885,1,5470.0,3,10,65.0,59.0,15.0


In [9]:
labels = df["IsClick"]
labels.head()

0    1
1    0
2    0
3    1
4    1
Name: IsClick, dtype: int64

In [10]:
inputs.describe()

,Position,HistCTR,IsUserLoggedOn,Price,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
count,5.766142e+06,5.766142e+06,5.766142e+06,5.764383e+06,5.766142e+06,5.766142e+06,5.764383e+06,2.669083e+06,900842.000000
mean,3.605067e+00,1.150275e-02,3.057816e-01,1.874836e+04,2.820993e+00,1.453817e+01,5.108515e+01,4.914596e+01,17.208384
std,2.973891e+00,1.618328e-02,4.607377e-01,1.787216e+05,2.048649e+00,5.752825e+00,1.989212e+01,2.421779e+01,9.731279
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+01,1.000000
25%,1.000000e+00,2.769000e-03,0.000000e+00,1.349000e+03,1.000000e+00,1.100000e+01,3.700000e+01,3.200000e+01,10.000000
50%,1.000000e+00,6.204000e-03,0.000000e+00,4.399000e+03,3.000000e+00,1.500000e+01,5.200000e+01,4.600000e+01,14.000000
75%,7.000000e+00,1.333300e-02,1.000000e+00,1.330000e+04,5.000000e+00,1.900000e+01,6.600000e+01,6.100000e+01,22.000000
max,7.000000e+00,1.000000e+00,1.000000e+00,1.300000e+08,6.000000e+00,2.300000e+01,9.700000e+01,1.980000e+02,162.000000


In [11]:
median_Price = inputs.Price.median()
inputs.Price.fillna(median_Price, inplace=True)

median_AdTitleSZ = inputs.AdTitleSZ.median()
inputs.AdTitleSZ.fillna(median_AdTitleSZ, inplace=True)

median_SearchParamsSZ = inputs.SearchParamsSZ.median()
inputs.SearchParamsSZ.fillna(median_SearchParamsSZ, inplace=True)

#maybe try zero
median_SearchQuerySZ = inputs.SearchQuerySZ.median()
inputs.SearchQuerySZ.fillna(median_SearchQuerySZ, inplace=True)

In [7]:
inputs.describe()

,Position,HistCTR,IsUserLoggedOn,Price,SearchCategoryLevel,reg_ads,highlighted_ads,context_ads,user_clicks_no,user_non_clicks_no,user_click_prob,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ,weight,IsClick
count,5.150998e+06,5.150998e+06,5.150998e+06,5.149593e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06,5.150998e+06
mean,3.594031e+00,1.125062e-02,3.144672e-01,1.842662e+04,2.693287e+00,1.702706e+00,3.985022e-01,1.863855e+00,2.537840e+00,1.437796e+01,1.966040e-01,2.867462e+00,1.459326e+01,5.101086e+01,2.292443e+01,2.613030e+00,3.321266e+01,1.966040e-01
std,2.972405e+00,1.593798e-02,4.643034e-01,9.229302e+04,5.958805e-01,1.303716e+00,5.079587e-01,3.429443e-01,3.858880e+00,2.413330e+01,1.980426e-01,1.977061e+00,5.775873e+00,1.994528e+01,2.966364e+01,7.221760e+00,2.325436e+01,3.974304e-01
min,1.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.000000e+00,2.715000e-03,0.000000e+00,1.300000e+03,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,1.000000e+00,1.100000e+01,3.600000e+01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,1.000000e+00,6.100000e-03,0.000000e+00,4.290000e+03,3.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,6.000000e+00,1.578947e-01,3.000000e+00,1.500000e+01,5.200000e+01,0.000000e+00,0.000000e+00,5.000000e+01,0.000000e+00
75%,7.000000e+00,1.299900e-02,1.000000e+00,1.330000e+04,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,1.600000e+01,3.333333e-01,5.000000e+00,1.900000e+01,6.600000e+01,4.200000e+01,0.000000e+00,5.000000e+01,0.000000e+00
max,7.000000e+00,1.000000e+00,1.000000e+00,1.040000e+07,3.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,1.070000e+02,5.960000e+02,1.000000e+00,6.000000e+00,2.300000e+01,9.700000e+01,1.980000e+02,1.620000e+02,5.000000e+01,1.000000e+00


In [63]:
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
inputs.Price = scaler.fit_transform(inputs.Price.reshape(-1, 1))
inputs.Position = scaler.fit_transform(inputs.Position.reshape(-1, 1))
inputs.day_of_week = scaler.fit_transform(inputs.day_of_week.reshape(-1, 1))
inputs.hour = scaler.fit_transform(inputs.hour.reshape(-1, 1))
inputs.AdTitleSZ = scaler.fit_transform(inputs.AdTitleSZ.reshape(-1, 1))
inputs.SearchParamsSZ = scaler.fit_transform(inputs.SearchParamsSZ.reshape(-1, 1))
inputs.SearchQuerySZ = scaler.fit_transform(inputs.SearchQuerySZ.reshape(-1, 1))

# scaler = preprocessing.StandardScaler()
# inputs.Price = scaler.fit_transform(inputs.Price)

In [64]:
inputs.describe()

,Position,HistCTR,IsUserLoggedOn,Price,day_of_week,hour,AdTitleSZ,SearchParamsSZ,SearchQuerySZ
count,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06,5.766142e+06
mean,4.341778e-01,1.150275e-02,3.057816e-01,1.441845e-04,4.701655e-01,6.320942e-01,6.305251e-01,1.340657e-01,7.330037e-02
std,4.956485e-01,1.618328e-02,4.607377e-01,1.374573e-03,3.414414e-01,2.501228e-01,2.566595e-01,7.839234e-02,2.104298e-02
min,0.000000e+00,1.000000e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.769000e-03,0.000000e+00,1.038462e-05,1.666667e-01,4.782609e-01,4.375000e-01,1.200000e-01,7.070707e-02
50%,0.000000e+00,6.204000e-03,0.000000e+00,3.383846e-05,5.000000e-01,6.521739e-01,6.458333e-01,1.200000e-01,7.070707e-02
75%,1.000000e+00,1.333300e-02,1.000000e+00,1.023077e-04,8.333333e-01,8.260870e-01,8.541667e-01,1.200000e-01,7.070707e-02
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [12]:
inputs.dtypes

Position            int64
HistCTR           float64
IsUserLoggedOn      int64
Price             float64
day_of_week         int64
hour                int64
AdTitleSZ         float64
SearchParamsSZ    float64
SearchQuerySZ     float64
dtype: object

In [13]:
inputs['Position'] = inputs['Position'].astype(float)
inputs['IsUserLoggedOn'] = inputs['IsUserLoggedOn'].astype(float)
inputs['day_of_week'] = inputs['day_of_week'].astype(float)
inputs['hour'] = inputs['hour'].astype(float)

In [14]:
inputs.dtypes

Position          float64
HistCTR           float64
IsUserLoggedOn    float64
Price             float64
day_of_week       float64
hour              float64
AdTitleSZ         float64
SearchParamsSZ    float64
SearchQuerySZ     float64
dtype: object

## Logistic Regression with Stochastic Gradient Descent

In [19]:
CLF_LogReg_SGF = SGDClassifier(loss="log")
CLF_LogReg_SGF.fit(inputs, labels)
print(CLF_LogReg_SGF)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)


In [21]:
CLF_LogReg_SGF.predict_proba(inputs)
CLF_LogReg_SGF.score(inputs, labels)

0.80171456061956159

## XGboost

In [35]:
CLF_Xgboost = xgboost.XGBClassifier()
CLF_Xgboost.fit(inputs, labels)
print(CLF_Xgboost)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


In [36]:
CLF_Xgboost.predict(inputs)
CLF_Xgboost.score(inputs, labels)

0.80597685592897295

## Vowpal Wabbit

In [37]:
CLF_wabbit = VWClassifier()
CLF_wabbit.fit(inputs, labels)
print(CLF_wabbit)

{'quiet': True, 'loss_function': 'logistic'}

In [38]:
# CLF_wabbit.predict(inputs)
CLF_wabbit.score(inputs, labels)

0.13603012204694231